In [8]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import numpy as np

# Importing Gensim
import gensim
from gensim import corpora

doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]


stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]   

print(doc_clean)

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)
print('before filter', dictionary)
dictionary.filter_extremes(no_below=2, no_above=0.8)
print('after filter', dictionary)
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
for doc in doc_clean:
    print(dictionary.doc2bow(doc))

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50, dtype=np.float64)

print('\n')
print(ldamodel.print_topics(num_topics=3, num_words=3))

[['sugar', 'bad', 'consume', 'sister', 'like', 'sugar', 'father'], ['father', 'spends', 'lot', 'time', 'driving', 'sister', 'around', 'dance', 'practice'], ['doctor', 'suggest', 'driving', 'may', 'cause', 'increased', 'stress', 'blood', 'pressure'], ['sometimes', 'feel', 'pressure', 'perform', 'well', 'school', 'father', 'never', 'seems', 'drive', 'sister', 'better'], ['health', 'expert', 'say', 'sugar', 'good', 'lifestyle']]
before filter Dictionary(35 unique tokens: ['bad', 'consume', 'father', 'like', 'sister']...)
after filter Dictionary(5 unique tokens: ['father', 'sister', 'sugar', 'driving', 'pressure'])
[(0, 1), (1, 1), (2, 2)]
[(0, 1), (1, 1), (3, 1)]
[(3, 1), (4, 1)]
[(0, 1), (1, 1), (4, 1)]
[(2, 1)]


[(0, '0.355*"driving" + 0.206*"pressure" + 0.194*"sister"'), (1, '0.275*"father" + 0.275*"sister" + 0.262*"sugar"'), (2, '0.505*"sugar" + 0.124*"pressure" + 0.124*"sister"')]
